In [1]:
# Libraries for general purpose
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import rotate, resize
import os 
from sklearn.model_selection import train_test_split
import torchvision.models as models

C:\Users\benal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\benal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
folder_path = r"C:\Users\benal\xray_dataset_covid19\train\NORMAL" 
train_img = []
train_img_values = []
train_y=[]
for filename in os.listdir(folder_path):
    image_path = folder_path+'/' + filename 
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    train_img.append(img)
    train_y.append(1)
train_x1 = np.array(train_img)
train_y1=np.array(train_y)

In [4]:
folder_path = r"C:\Users\benal\xray_dataset_covid19\train\PNEUMONIA" 
train_img = []
train_img_values = []
train_y=[]
for filename in os.listdir(folder_path):
    image_path = folder_path+'/' + filename 
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    train_img.append(img)
    train_y.append(0)
train_x2 = np.array(train_img)
train_y2=np.array(train_y)

In [5]:
train_x = np.concatenate((train_x1, train_x2), axis=0)
train_y = np.concatenate((train_y1, train_y2), axis=0)
random_indices = np.random.permutation(train_x.shape[0])
train_x = train_x[random_indices]
train_y = train_y[random_indices]


In [9]:
folder_path = r"C:\Users\benal\xray_dataset_covid19\test\NORMAL" 
test_img = []
test_img_values = []
test_y=[]
for filename in os.listdir(folder_path):
    image_path = folder_path+'/' + filename 
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    test_img.append(img)
    test_y.append(1)
test_x1 = np.array(test_img)
test_y1=np.array(test_y)

In [11]:
folder_path = r"C:\Users\benal\xray_dataset_covid19\test\PNEUMONIA" 
test_img = []
test_img_values = []
test_y=[]
for filename in os.listdir(folder_path):
    image_path = folder_path+'/' + filename 
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    test_img.append(img)
    test_y.append(0)
test_x2 = np.array(test_img)
test_y2=np.array(test_y)

In [12]:
final_train_x = []
final_train_y = []

#train_x.shape[0]
for i in range(train_x.shape[0]):
    final_train_x.append(train_x[i])
    final_train_x.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_x.append(np.fliplr(train_x[i]))
    for j in range(3):
      final_train_y.append(train_y[i])

train_x = np.array(final_train_x)
train_y = np.array(final_train_y)

In [14]:
test_x = np.concatenate((test_x1, test_x2), axis=0)
test_y = np.concatenate((test_y1, test_y2), axis=0)
random_indices = np.random.permutation(test_x.shape[0])
test_x = test_x[random_indices]
test_y =test_y[random_indices]

In [15]:
final_test_x = []
final_test_y = []

#train_x.shape[0]
for i in range(test_x.shape[0]):
    final_test_x.append(test_x[i])
    final_test_x.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_test_x.append(np.fliplr(train_x[i]))
    for j in range(3):
      final_train_y.append(train_y[i])

test_x = np.array(final_test_x)
test_y = np.array(final_test_y)

In [16]:
import torch 
train_x = train_x.reshape(train_x.shape[0], 3, 224, 224)  
train_x  = torch.from_numpy(train_x)
train_x = train_x.float()
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)
 
test_x = test_x.reshape(test_x.shape[0], 3, 224, 224)  
test_x  = torch.from_numpy(test_x)
test_x = test_x.float()
test_y = test_y.astype(int)
test_y = torch.from_numpy(test_y)


In [23]:
import torch.optim as optim
import torch.nn as nn
def CNN_Model(pretrained=True):
    model = models.densenet121(pretrained=pretrained) # Returns Defined Densenet model with weights trained on ImageNet
    num_ftrs = model.classifier.in_features # Get the number of features output from CNN layer
    model.classifier = nn.Linear(num_ftrs, 2) # Overwrites the Classifier layer with custom defined layer for transfer learning
    model = model.to(device) # Transfer the Model to GPU if available
    return model

model = CNN_Model(pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler


In [24]:
batch_size = 32
n_epochs = 1

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    permutation = torch.randperm(train_x.size()[0])
    training_loss = []
    for i in range(0,train_x.size()[0], batch_size):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y.long())

        loss.backward()
        optimizer.step() 
        
    

In [25]:
from sklearn.metrics import accuracy_score

permutation = torch.randperm(test_x.size()[0])
y_pred_list = []
y_test_list=[]
for i in range(0,test_x.size()[0], batch_size):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        output = model(batch_x)
        y_pred_test = torch.argmax(output, dim=1)
        y_pred_list.extend(y_pred_test.squeeze().tolist())
        y_test_list.extend(batch_y.squeeze().tolist())
        
accuracy = accuracy_score(y_test_list, y_pred_list)
print(f"Accuracy: {accuracy}")
         

Accuracy: 1.0
